In [ ]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
from pypardiso import spsolve
from pathlib import Path
from gsa_framework.utils import read_pickle, write_pickle
from gsa_framework.visualization.plotting import plot_histogram_Y1_Y2
from fs.zipfs import ZipFS
from copy import deepcopy
from consumption_model_ch.utils import get_habe_filepath
import pandas as pd
import sys
sys.path.append('/Users/akim/PycharmProjects/akula')
from collections import Counter

from akula.sensitivity_analysis import get_mask

project = 'GSA for archetypes'
bd.projects.set_current(project)
iterations = 2000
seed = 11111000 

fp_paper_2 = Path("write_files") / project.lower().replace(" ", "_") / "paper_2"
fp_paper_2.mkdir(exist_ok=True, parents=True)

In [ ]:
# Foreground uncertainty weighted by number of people per household
def get_household_data_weighted(indices, co_name="swiss consumption 1.0"):
    # 1. Get some metadata from the consumption database
    co = bd.Database(co_name)
    year_habe = co.metadata['year_habe']
    dir_habe = co.metadata['dir_habe']

    # 2. Extract total demand from HABE
    path_ausgaben = get_habe_filepath(dir_habe, year_habe, 'Ausgaben')
    path_mengen = get_habe_filepath(dir_habe, year_habe, 'Mengen')
    path_konsumgueter = get_habe_filepath(dir_habe, year_habe, 'Konsumgueter')
    path_personen = get_habe_filepath(dir_habe, year_habe, 'Personen')

    # change codes to be consistent with consumption database and Andi's codes
    ausgaben = pd.read_csv(path_ausgaben, sep='\t')
    mengen = pd.read_csv(path_mengen, sep='\t')
    konsumgueter = pd.read_csv(path_konsumgueter, sep='\t')
    personen_raw = pd.read_csv(path_personen, sep='\t')
    ausgaben.columns = [col.lower() for col in ausgaben.columns]
    mengen.columns = [col.lower() for col in mengen.columns]
    konsumgueter.columns = [col.lower() for col in konsumgueter.columns]
    personen_raw.columns = [col.lower() for col in personen_raw.columns]
    num_personen = dict(Counter(personen_raw["haushaltid"]))
    num_personen = [{'haushaltid': k, "n_personen": v} for k,v in num_personen.items()]
    personen = pd.DataFrame.from_dict(num_personen)

    codes_co_db = sorted([act['code'] for act in co])
    columns_a = ausgaben.columns.values
    columns_m = [columns_a[0]]
    columns_k = [columns_a[0]]
    codes_m = []
    for code_a in columns_a[1:]:
        code_m = code_a.replace('a', 'm')
        if code_m in codes_co_db:
            columns_m.append(code_m)
            codes_m.append(code_m)
        else:
            columns_m.append(code_a)
    ausgaben.columns = columns_m
    # Replace ausgaben data with mengen data
    for code_m in codes_m:
        ausgaben[code_m] = mengen[code_m].values
    weighted_ausgaben = pd.concat(
        [
            personen.set_index('haushaltid'),
            ausgaben.set_index('haushaltid'), 
        ],
        join='inner',
        axis=1
    )
    weighted_ausgaben = weighted_ausgaben.iloc[:,1:].div(weighted_ausgaben['n_personen'], axis=0)
    weighted_ausgaben = weighted_ausgaben.reset_index()

    data = np.zeros((0,len(weighted_ausgaben)))
    for inds in indices:
        input_code = bd.get_activity(inds[0])['code']
        try:
            data_current = weighted_ausgaben[input_code].values
            data = np.vstack([data, data_current])
        except:
            print(input_code)

    return data

In [ ]:
bd.projects.set_current("GSA for archetypes")
co_name = "swiss consumption 1.0"
co = bd.Database(co_name)

In [ ]:
a = co.random()
list(a.exchanges())[1].as_dict()

In [ ]:
data = ausgaben['a30'].values

In [ ]:
from scipy.stats import lognorm

In [ ]:
lognorm.fit??

In [ ]:
import plotly.graph_objects as go

In [ ]:
x = np.linspace(min(data), max(data), 10000)
pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
       / (x * sigma * np.sqrt(2 * np.pi)))

In [ ]:
num_bins = 200
bins_ = np.linspace(min(data), max(data), num_bins, endpoint=True)
freq, bins = np.histogram(data, bins=bins_)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=bins,
        y=freq,
        opacity=0.60,
        showlegend=True,
    ),
)

In [ ]:
bd.projects.set_current("GSA for archetypes")
co_name = "swiss consumption 1.0"

co = bd.Database(co_name)
year_habe = co.metadata['year_habe']
dir_habe = co.metadata['dir_habe']